## Illustration of **wandb** for visual inspection og ML optimization and validation

- Sign-up at https://wandb.ai/ 
- Install wandb in the same virtual environment as tensorflow
  - conda install -c conda-forge wandb
- in the prompt: wandb login
  - copy the key from the web page and paste it in the prompt

In [10]:
import os
import wandb

from wandb.keras import WandbCallback

In [11]:
import tensorflow as tf

from matplotlib import pyplot as plt

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = x_train / 255
x_test = x_test / 255

In [12]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [13]:
def build_model(size, activation, weight_regularizer, dropout, batch_norm):
    layers = [tf.keras.layers.Flatten(input_shape=(28, 28))]
    
    nb_layers = size
    nb_neurons = size ** 2 * 4
    
    for i in range(nb_layers):
        layers.append(tf.keras.layers.Dense(nb_neurons, activation=activation, 
                                            kernel_regularizer=weight_regularizer))
        if batch_norm:
            layers.append(tf.keras.layers.BatchNormalization())
        if dropout:
            layers.append(tf.keras.layers.Dropout(0.2))

    layers.append(tf.keras.layers.Dense(10, activation='softmax'))
    
    model = tf.keras.models.Sequential(layers)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.0001),
        metrics=['sparse_categorical_crossentropy', 'accuracy'],
        )
    
    return model

In [14]:
model_small = build_model(1, 'relu', None, False, False)
model_small.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 3140      
                                                                 
 dense_3 (Dense)             (None, 10)                50        
                                                                 
Total params: 3,190
Trainable params: 3,190
Non-trainable params: 0
_________________________________________________________________


In [15]:
## Here you need to supply your own wandb API key (if it has not been set by wandb login at prompt has not been used), project name and entity name.

os.environ["WANDB_API_KEY"]= 'dab3e100176ff8e1ab9c3aee6b30924031c8cde6'
os.environ["WANDB_NAME"]= 'fashion-mnist-small'
os.environ["WANDB_NOTEBOOK_NAME"]='TryWandb.ipynb'
run = wandb.init(project="my-awesome-project",entity='theboys233',job_type='train')
config = wandb.config

In [16]:
print("TensorFlow Version:", tf.__version__)

# Check available GPUs
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

TensorFlow Version: 2.9.1
Num GPUs Available:  1
Name: /physical_device:GPU:0   Type: GPU


In [17]:
# Define WandbCallback for experiment tracking
wandb_callback = WandbCallback(monitor='val_loss',verbose=True,save_model=False,
                               log_weights=True,
                               log_evaluation=True,
                               validation_steps=2)
callbacks = [wandb_callback]


In [18]:
history_small = model_small.fit(x_train, y_train,
                                validation_data=(x_test, y_test),
                                epochs=40, verbose=1,
                                callbacks = callbacks)

Epoch 1/40
1875/1875 [==============================] - 7s 3ms/step - loss: 1.9738 - sparse_categorical_crossentropy: 1.9738 - accuracy: 0.2600 - val_loss: 1.7357 - val_sparse_categorical_crossentropy: 1.7357 - val_accuracy: 0.3517
Epoch 2/40
1875/1875 [==============================] - 6s 3ms/step - loss: 1.5298 - sparse_categorical_crossentropy: 1.5298 - accuracy: 0.4484 - val_loss: 1.2179 - val_sparse_categorical_crossentropy: 1.2179 - val_accuracy: 0.5839
Epoch 3/40
1875/1875 [==============================] - 6s 3ms/step - loss: 1.0255 - sparse_categorical_crossentropy: 1.0255 - accuracy: 0.6532 - val_loss: 0.9228 - val_sparse_categorical_crossentropy: 0.9228 - val_accuracy: 0.6822
Epoch 4/40
1875/1875 [==============================] - 6s 3ms/step - loss: 0.8564 - sparse_categorical_crossentropy: 0.8564 - accuracy: 0.7070 - val_loss: 0.8294 - val_sparse_categorical_crossentropy: 0.8294 - val_accuracy: 0.7175
Epoch 5/40
1875/1875 [==============================] - 5s 3ms/step - lo